In [ ]:
!pip -q install --upgrade diffusers transformers accelerate safetensors
!pip -q install fastapi uvicorn pyngrok pillow python-multipart nest_asyncio


In [ ]:
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

model_id = "stabilityai/stable-diffusion-2-1"
# model_id = "runwayml/stable-diffusion-v1-5"  # reliable on free Colab
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    safety_checker=None,
)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")


In [ ]:
import base64, io
from PIL import Image
from pyngrok import conf
conf.get_default().auth_token = "31VNKn4EB0ohx8g8nj5FgVPDZ1V_78VUNwqptBEWt4EMQXynT"

def sd15_b64(prompt, negative=None, width=512, height=512, steps=25, guidance=7.5, seed=None):
    g = torch.Generator("cuda").manual_seed(seed) if seed is not None else None
    img = pipe(
        prompt,
        negative_prompt=negative,
        width=width, height=height,
        num_inference_steps=steps,
        guidance_scale=guidance,
        generator=g,
    ).images[0]
    buf = io.BytesIO(); img.save(buf, format="PNG")
    return base64.b64encode(buf.getvalue()).decode()


In [ ]:
import nest_asyncio, uvicorn
from pyngrok import ngrok
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

# Optional: if you have an ngrok account, uncomment and paste your token for a steadier URL
# from pyngrok import conf; conf.get_default().auth_token = "NGROK_TOKEN"

class GenerateReq(BaseModel):
    text: str
    negative_prompt: str | None = None
    width: int | None = None
    height: int | None = None
    steps: int | None = None
    guidance: float | None = None
    seed: int | None = None

app = FastAPI(title="Colab SD1.5 API")
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], allow_methods=["*"], allow_headers=["*"]
)

@app.get("/health")
def health(): return {"status":"ok","model":"runwayml/stable-diffusion-v1-5"}

@app.post("/generate")
def generate(p: GenerateReq):
    b64 = sd15_b64(
        prompt=p.text,
        negative=p.negative_prompt,
        width=p.width or 512, height=p.height or 512,
        steps=p.steps or 25, guidance=p.guidance or 7.5,
        seed=p.seed,
    )
    return {"image": b64}

public_url = ngrok.connect(8000, bind_tls=True).public_url
print("PUBLIC API URL:", public_url)

nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)


PUBLIC API URL: https://53f896ae19f0.ngrok-free.app


INFO:     Started server process [276]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     2a02:8108:96ba:c000:64dc:4fbe:1e15:eb4f:0 - "OPTIONS /generate HTTP/1.1" 200 OK


  0%|          | 0/25 [00:00<?, ?it/s]

INFO:     2a02:8108:96ba:c000:64dc:4fbe:1e15:eb4f:0 - "POST /generate HTTP/1.1" 200 OK


...

INFO:     2a02:8108:96ba:c000:64dc:4fbe:1e15:eb4f:0 - "POST /generate HTTP/1.1" 200 OK


  0%|          | 0/25 [00:00<?, ?it/s]

INFO:     2a02:8108:96ba:c000:64dc:4fbe:1e15:eb4f:0 - "POST /generate HTTP/1.1" 200 OK


  0%|          | 0/25 [00:00<?, ?it/s]

INFO:     2a02:8108:96ba:c000:64dc:4fbe:1e15:eb4f:0 - "POST /generate HTTP/1.1" 200 OK
